<a href="https://colab.research.google.com/github/abala2017/SalesBot/blob/main/Build.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install openai

In [ ]:
import openai
import tensorflow as tf
import pandas as pd
import random
import numpy as np
import tkinter as tk
from tkinter import *
from sklearn.model_selection import train_test_split

In [ ]:
def getPrices():
  df = pd.read_csv("productData.csv")
  X = pd.get_dummies(df["type"],prefix = "type")
  X = pd.concat([X,pd.get_dummies(df["brand"], prefix="brand")],axis=1)
  X = pd.concat([X,pd.get_dummies(df["season"],prefix = "season")],axis=1)
  Y = df["sales"].apply(encodeY)
  Y = pd.get_dummies(Y)
  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
  model = tf.keras.Sequential(
      [
       tf.keras.layers.Dense(64, input_dim = 12, activation='relu'),
       tf.keras.layers.Dense(32, activation='relu'),
       tf.keras.layers.Dense(4, activation='softmax')
      ]
      )
  model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
  model.build()
  _ = model.fit(x=X_train, y = y_train,epochs=50, validation_data=(X_test,y_test),verbose=0)
  df = pd.read_csv("initialProductData.csv")
  X = pd.get_dummies(df["type"],prefix = "type")
  X = pd.concat([X,pd.get_dummies(df["brand"], prefix="brand")],axis=1)
  X["Spring"] = 0
  X["Summer"] = 0
  X["Fall"] = 0
  X["Winter"] = 1
  Y = model.predict(X)
  Y = np.argmax(Y,axis=1)
  f = open("botProductData.jsonl","w")
  for i in df.index:
    discount = 1
    if(Y[i] == 3):
      discount = 1
    if(Y[i] == 2):
      discount = .8
    if(Y[i] == 1):
      discount = .7
    if(Y[i] == 0):
      discount = .6
    f.write('{"text":"product: '+str(df.loc[i,"product"])+', price: ' +str(df.loc[i,"price"] * discount)+', colors : '+ df.loc[i,"colors"] +', description : ' + df.loc[i,"desciption"]+'"} \n')
  f.close()

In [ ]:
def encodeY(val):
  if(val < 2000):
    return 0
  if(val >= 2000 and val < 4000):
    return 0
  if(val >= 4000 and val < 6000):
    return 1
  if(val >= 6000 and val < 8000):
    return 1
  if(val >= 8000 and val < 10000):
    return 2
  if(val >= 10000 and val < 12000):
    return 2
  if(val >= 12000 and val < 14000):
    return 2
  if(val >= 14000 and val < 16000):
    return 3
  if(val >= 16000 and val < 18000):
    return 3
  if(val >= 18000):
    return 3

In [ ]:
def salesBot():
  openai.api_key = "sk-0alrPzzVbaB1jFTsA2T2LqgXMnA2W9pC4jN8ukaE"
  response = openai.File.create(
  file=open("/content/botProductData.jsonl"),
  purpose='answers'
  )
  inputStr = ""
  while(inputStr != "thanks" and inputStr != "Thanks"):
    inputStr = input("How can I help you? ")
    try:
      ans = openai.Answer.create(
          search_model="ada", 
          model="curie",
          question=inputStr, 
          file = response.id,
          ##documents = [],
          examples_context="In 2017, U.S. life expectancy was 78.6 years.", 
          examples=[["What is human life expectancy in the United States?","78 years."]],
          max_tokens=5,
          stop=["\n", "<|endoftext|>"],
          )
    except:
      ans = openai.Answer.create(
          search_model="ada", 
          model="curie",
          question=inputStr, 
          ##file = response.id,
          documents = [],
          examples_context="In 2017, U.S. life expectancy was 78.6 years.", 
          examples=[["What is human life expectancy in the United States?","78 years."]],
          max_tokens=5,
          stop=["\n", "<|endoftext|>"],
          )
    print(ans.answers[0])

Main Function

In [ ]:
getPrices()

In [ ]:
salesBot()

Upload progress: 100%|██████████| 2.41k/2.41k [00:00<00:00, 3.94kit/s]
